## 0. library

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

## 16번 데이터 불러오기

In [2]:
dat_16 = gpd.read_file("16.용인시_소상공인_매출정보.geojson")
dat_16 = dat_16.drop([9914],axis=0) ## 9914의 index에 있는 값은 포천의 좌표라서 제거
dat_16['ws_cnt'] = dat_16['ws_cnt'].fillna(0)

## 1.주거 건물수 변수 제작 - 결과물 : house_counts

In [3]:
## Multipolygon 형식을 list 형식으로 바꾸는 함수
def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

## 여러 좌표의 중심점을 도출하는 함수
def center_point(x):
    first = np.mean([w[0] for w in x])
    second = np.mean([w[1] for w in x])
    return [first, second]


############################ 1. 데이터 불러오기 ####################################

dat_8 = gpd.read_file('8.용인시_도로명주소_건물.geojson', encoding='utf-8')

############################ 2. 8번 건물 데이터 전처리 ####################################

## 건물 데이터 multipoloygon을 list 형식으로 바꾸기 + 건물 중심좌표 만들기

dat_8['coordinates'] = dat_8['geometry'].apply(multipolygon_to_coordinates)
dat_8['center'] = dat_8['coordinates'].apply(center_point)


############################ 3. 16번 좌표 데이터 전처리 ####################################

## Multipolygon을 list로 전환

dat_16['coordinates'] = dat_16['geometry'].apply(multipolygon_to_coordinates)

## 위도별 경도별로 list 분해하는 작업

def lat_lon(x):
    first = [w[0] for w in x]
    second = [w[1] for w in x]
    return np.array([first,second])

dat_16['lat_lon'] = dat_16['coordinates'].apply(lat_lon)

############################ 4. 건물 좌표를 16번의 gid에 배정하기 ####################################

## 16번 데이터의 gid의 범위를 설정하기 위한 aray 4개 만들기

lat_lon = dat_16['lat_lon'].values
min_1 = np.array([min(w[0]) for w in lat_lon]) # 모든 gid 내 경도에 대한 최소값
max_1 = np.array([max(w[0]) for w in lat_lon]) # 모든 gid 내 경도에 대한 최대값
min_2 = np.array([min(w[1]) for w in lat_lon]) # 모든 gid 내 위도에 대한 최소값
max_2 = np.array([max(w[1]) for w in lat_lon]) # 모든 gid 내 위도에 대한 최대값

## target : (x,y) 형식으로 이루어진 좌표값 (건물의 중심 좌표)
## min / max : 16번 좌표 데이터의 최대/최소의 위도/경도 값들 (앞서 구한 array들)
## loc : array([index]) 형태로 각 건물의 중심 좌표가 몇 번째 gid에 배정되는지를 나타내줌 (ireturn 값은 index 형태)

def mapping_gid(target, min_1=min_1, max_1=max_1, min_2=min_2,max_2=max_2):
    X = target[0]
    y = target[1]
    loc = np.where(( min_1<=X) & (max_1>=X) & (min_2<=y) & (max_2>=y))[0][0]
    return loc

## 건물 데이터 중심 좌표 불러오기
build_points = dat_8['center'].tolist()

## 건물 중심 좌표가 어느 gid에 속해있는지 mapping 하기
new_build_points = list(map(mapping_gid,build_points)) 

## 계산된 index 값들을 다시 gid 값으로 전환
gid_list = dat_16['gid'].values
gid_match = [gid_list[w] for w in new_build_points] ## w가 index값이기에 실제 gid 값으로 반환됨
dat_8['gid'] = gid_match ## 병합 완료

############################ 5. 격자별 인접 격자 구하기 ####################################

## 격자별 중심점 구하는 함수

def center_point(x):
    first = np.mean([w[0] for w in x[0:4]]) ## coordinates가 시작점~시작점으로 끝남으로 4개의 점만 사용
    second = np.mean([w[1] for w in x[0:4]])
    return (first, second)

## 격자 중심 변수 제작

dat_16['center'] = dat_16['coordinates'].apply(center_point)
points_vector = dat_16['center'].tolist()

## 위도 경도 min_max scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
points_vector = scaler.fit_transform(points_vector)

## distance matrix 구성하기
from scipy.spatial import distance
dist_mat = distance.cdist(points_vector, points_vector, 'euclidean')

## 인접 격자는 최대 8개까지임으로 8개의 인접 좌표 추출 
close_8_dist = [np.sort(w)[1:9] for w in dist_mat]

## 하지만 용인시 격자를 시각화해보면 용인시 경계선에 위치한 격자들은 인접 격자가 8개가 아니고 위치에 따라 다름
## 따라서 이를 계산하기 위해서 인접 격자들간의 거리를 평균을 한 다음 histogram을 통해 적절한 cutpoint을 찾음
mean_8_dist = [np.mean(w) for w in close_8_dist]

threshold_dist = 0.011 ## 이 숫자보다 크면 주변 칸(점을 맞대고 있는 지점)을 벗어난다고 가정
under_threshold_dist = [w[np.where(w <= threshold_dist)] for w in close_8_dist]

## 한 격자에 대해 거리가 가장 가까운 8개의 격자 중 CUTPOINT를 넘어서는 것들을 제외한 격자만 추출
## 제대로 인접한 격자를 찾는 과정
neighbors_points = [[np.where(dist_mat[index] == w)[0][0] for w in under_threshold_dist[index]] for index in range(len(under_threshold_dist))]

## gid를 배정해주기 
gid_vector = dat_16['gid'].values
neighbors_gid = [list(gid_vector[w]) for w in neighbors_points]
dat_16['neighbors'] = neighbors_gid ## 각 gid마다 주변 gid가 무엇인지 나타내는 변수

############################ 6. 인접 격자 포함 주거 건물 개수 구하기 ####################################

housing = ['01000','01001','01002','01003','01004','02000','02001','02002','02003','02004','02005','02006','02007'] ## 주거건물코드
dat_8['house'] = dat_8['BDTYP_CD'].apply(lambda x: "Y" if x in housing else None) ## 주거 건물만을 FILTERING하기 위함
gid_house = dat_8[['gid','house']].groupby(by='gid').count() ## gid 별로 주거 건물이 몇 개인지 count
gid_house_dict = gid_house.to_dict()['house'] ## "gid : house 개수" 형태의 dictionary 

## 최종적으로 gid별 주거건물 개수 (주변 250m 범위) 추가해주기
## 인접 격자들의 주거 건물 개수를 다 더하여 해당 격자에 배정
def dict_identifier(target, dictionary):
    try:
        return dictionary[target]
    except:
        return 0

counts = [sum([dict_identifier(j,gid_house_dict) for j in w]) for w in neighbors_gid]
dat_16['house_counts'] = counts ## 변수 저장

## 2. 인구 변수 제작 - 결과물 : population

In [4]:
############################ 1. 데이터 불러오기 ####################################

dat_3 = gpd.read_file('3.용인시_인구정보(총인구수)_격자.geojson', encoding='utf-8')

############################ 2. 3번 건물 데이터 전처리 ####################################

## Multipolygon 형식을 list 형식으로 수정
dat_3['coordinates'] = dat_3['geometry'].apply(multipolygon_to_coordinates)
dat_3['val'] = dat_3['val'].fillna(0) ## NA인 경우 0을 넣어 처리

############################ 3. 인구 격자 좌표를 16번의 gid에 배정하기 ####################################

## 인구 격자는 50*50이며 16번의 데이터는 250*250이다. 따라서 격자 안에 포함될 수도, 두 격자 사이에 겹칠 수도
## 격자의 경계선에 놓이는 등 다양한 경우의 수가 존재한다.
## 따라서 추후 계산을 쉽게하기 위해서 먼저 인구 격자(50*50)이 어느 16번 격자와 접점이 있는지 확인한다.
## 만약 포함관계면 1개만 나올 것이고, 겹쳐져 있다면 2~4개로 나올 것이다.

## HOUSE_COUNTS과 동일한 과정 진행
min_1 = np.array([min(w[0]) for w in lat_lon])
max_1 = np.array([max(w[0]) for w in lat_lon])
min_2 = np.array([min(w[1]) for w in lat_lon])
max_2 = np.array([max(w[1]) for w in lat_lon])

def mapping_gid(target, min_1=min_1, max_1=max_1, min_2=min_2,max_2=max_2):
    result = []
    for element in target[0:4]:
        X = round(element[0],4)
        y = round(element[1],4)
        try:
            loc = np.where(( min_1<=X) & (max_1>=X) & (min_2<=y) & (max_2>=y))[0][0]
            result.append(loc)
        except:
            result.append("NA")
    return result

## 인구 데이터 coordinates 가져오기
popul_points = dat_3['coordinates'].tolist()

## 인구 데이터에 gid index 매칭하기
## 인구 격자의 4개 점을 기준으로 어느 GID에 속해있는지 확인하는 것이기 때문에 4개씩 나오게 됨
new_popul_points = list(map(mapping_gid,popul_points))

def unique_counts(x):
    T = np.unique(x)
    return T

## 우리는 그 중 어느 격자와 인구 격자가 관계가 있는지 확인하고 싶은 것이기에 UNIQUE값만 확인
uni_list = [unique_counts(w) for w in new_popul_points]

############################ 4. 교집합 면적을 이용하여 16번 데이터에 인구수 배정하기 ####################################

## 면적별로 계산해서 gid에 매칭하기
## 계산식 : 16번 데이터 격자 = 해당 격자 안에 있는 인구 격자에 대해서 비율*인구수를 더하기
## 비율 = 겹치는 면적 / 50*50M 면적

val_list = dat_3['val'].tolist() ## populaton의 value가 있는 list
gid_16 = [0 for i in range(len(dat_16))] ## gid 매칭을 위한 빈 리스트
popul_geo_array = dat_3['geometry'].values ## 인구 데이터 geometry array
geo_array = dat_16['geometry'].values ## 16번 gid 데이터 geometry array

### 

for tup in enumerate(uni_list):
    loc = tup[0]
    row = tup[1]
    if loc % 5000 == 0: ## 돌아가는 과정을 표시
        print(loc)
    if len(row) == 1:  ## 만약 unique값이 1개인 경우 그대로 population을 넣어줌
        index = row[0]
        
        if index != "NA":
            if gid_16[index] == 0: ## 만약 GID별 POPULATION값이 0이면 (아직 배정이 안되어있으면) 
                gid_16[index] = val_list[index] ## 새롭게 추가해줌
            else:
                gid_16[index] = gid_16[index] + val_list[index] ## 만약 0이 아니라면 기존에 있는 값에 더하여 UPDATE
        else:
            pass
                
    else: ## 만약 unique값이 1개가 아니라면, 각각의 값에 대해서 비중을 계산하여 인구수 배정해줌 
        target = popul_geo_array[loc]
        target_area = target.area ## 50*50 격자의 면적
        popul = val_list[loc]
        
        for index in row:
            if index != "NA":
                index = int(index)
                ith_val = popul * (geo_array[int(index)].intersection(target).area / target_area) ## (교집합/면적)*인구수
                
                if gid_16[index] == 0:
                    gid_16[index] = ith_val
                else:
                    gid_16[index] = gid_16[index] + ith_val
                
            else:
                pass


dat_16['population'] = gid_16

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000


## 3.결과물 저장하기

In [5]:
dat_16[['gid','house_counts','population']].to_csv("cluster_data.csv",index=False)